In [30]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import sys
# TODO: update this path
sys.path.append('/nfshomes/vla/low_bit_vision/lavis_clone/lavis')

# args parser for blip-2 runs
from args_parser import args_parser

import ast
from glob import glob
import re
import mmap

In [32]:
def load_result(file_path):
    with open(file_path, 'r+') as f:
        # read last line and grab metrics info as a dict
        result = ast.literal_eval(f.readlines()[-1].split('[INFO] ')[1])
        
        # grab model size output and add to results
        data = mmap.mmap(f.fileno(), 0)
        model_size = float(re.search(rb'\[Model Size\]: (.*)', data).group(1))
        result['model_size'] = model_size
        
    return result

In [33]:
# baseline result
baseline_result = load_result(os.path.join('..', 'results', 'blip2_flickr', 'blip2_flickr_baseline'))
baseline_result

{'txt_r1': 97.6,
 'txt_r5': 100.0,
 'txt_r10': 100.0,
 'txt_r_mean': 99.2,
 'img_r1': 89.74,
 'img_r5': 98.18,
 'img_r10': 98.94,
 'img_r_mean': 95.62,
 'r_mean': 97.41,
 'agg_metrics': 99.2,
 'model_size': 4782.180084}

In [34]:
parser = args_parser()
results_dir = os.path.join('..', 'results', 'blip2_flickr', 'uniform_quant')
results_dir

'../results/blip2_flickr/uniform_quant'

In [39]:
df_results = pd.DataFrame()

for folder in os.listdir(results_dir):
    folder_path = os.path.join(results_dir, folder)
    
    cli_args = []
    with open(os.path.join(folder_path, 'now.txt'), 'r') as f:
        for line in f:
            # skip the torch.distributed args
            params = line.split()[5:]
            args = vars(parser.parse_args(params))
            cli_args.append(args)
    
    gather = []
    # grab all files starting with a number (results)
    for result_path in glob(os.path.join(folder_path, '[0-9]*_log.txt')):
        
        file_name = os.path.basename(result_path)
        index = int(re.search(r'(\d)+', file_name).group(0))
        assert index != None
        
        result = load_result(result_path)
        result['index'] = index
        result['job_batch'] = folder
        
        gather.append(result)
    
    df_args = pd.DataFrame(cli_args)
    
    print(folder_path)
    # need to sort to match up with args from now.txt
    df_metrics = pd.DataFrame(gather)
    df_metrics = df_metrics.sort_values(by='index')
    df_metrics = df_metrics.set_index('index')
    
    df_result = pd.concat([df_args, df_metrics], axis = 1)
    df_results = pd.concat([df_results, df_result], axis = 0)


df_results = df_results.drop(['cfg_path', 'options'], axis = 1)

../results/blip2_flickr/uniform_quant/nbit_flickr_8_6
../results/blip2_flickr/uniform_quant/nbit_qformer
../results/blip2_flickr/uniform_quant/nbit_flickr_4_2
../results/blip2_flickr/uniform_quant/nbit_vit


In [40]:
# add baseline results and convert None values to np.nan
df_results = pd.merge(pd.DataFrame([baseline_result]),
                      df_results,
                      how = 'outer')

df_results = df_results.fillna(value=np.nan)

df_results.sort_values(by = 'model_size', ascending = False)

/tmp/ipykernel_4078795/3680435273.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_results = df_results.fillna(value=np.nan)


,txt_r1,txt_r5,txt_r10,txt_r_mean,img_r1,img_r5,img_r10,img_r_mean,r_mean,agg_metrics,...,qformer_img_ff_modules,qformer_img_ff_weight_bits,qformer_cls_modules,qformer_cls_transform_weight_bits,qformer_cls_decoder_weight_bits,output_modules,vision_proj_weight_bits,text_proj_weight_bits,itm_head_weight_bits,job_batch
654,97.6,100.0,100.0,99.200000,89.74,98.18,98.94,95.620000,97.410000,99.200000,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_qformer
647,97.6,100.0,100.0,99.200000,89.74,98.18,98.94,95.620000,97.410000,99.200000,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_qformer
649,97.6,100.0,100.0,99.200000,89.74,98.18,98.94,95.620000,97.410000,99.200000,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_qformer
650,97.6,100.0,100.0,99.200000,89.74,98.18,98.94,95.620000,97.410000,99.200000,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_qformer
651,97.6,100.0,100.0,99.200000,89.74,98.18,98.94,95.620000,97.410000,99.200000,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_qformer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,18.2,31.7,37.5,29.133333,11.68,24.20,29.52,21.800000,25.466667,29.133333,...,"[intermediate_query, output_query]",4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_flickr_4_2
64,0.1,0.5,0.9,0.500000,0.16,0.54,0.86,0.520000,0.510000,0.500000,...,"[intermediate_query, output_query]",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_flickr_4_2
71,0.1,0.5,1.0,0.533333,0.08,0.56,1.00,0.546667,0.540000,0.533333,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_flickr_4_2
12,0.0,0.4,0.8,0.400000,0.14,0.52,0.86,0.506667,0.453333,0.400000,...,"[intermediate_query, output_query]",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nbit_flickr_4_2


In [41]:
df_results.to_csv('blip2_flickr_results.csv', index = None)